# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file_path = r"C:\Users\admin\Desktop\2024\Data-Analyst\Assignments\Api_challenge\python_api_challenge\Starter_Code\WeatherPy\city_data.csv"
city_data_df = pd.read_csv(csv_file_path)

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,adamstown,-25.0660,-130.1015,20.71,78,100,9.34,PN,1721128522
1,tolanaro,-25.0319,46.9987,23.70,64,0,2.52,MG,1721128523
2,puerto baquerizo moreno,-0.9000,-89.6000,21.27,84,37,4.93,EC,1721128523
3,wailua homesteads,22.0669,-159.3780,24.24,81,100,7.20,US,1721128523
4,yellowknife,62.4560,-114.3525,18.29,68,75,6.71,CA,1721128523


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Step 1: Create a Map of Cities
# Configure the map plot
map_of_cities = city_data_df.hvplot.points(
    "Lng",                  # Longitude points
    "Lat",                  # Latitude points
    geo=True,               # Plot on a geographic coordinate system
    tiles="OSM",            # Use Open Street Maps
    color="City",           # Different colours for each city
    alpha=0.5,              # Sets Transparency at 50%
    size="Humidity",        # Size of spot determined by humidity
    scale=0.5,              # Scaling of the spot
    frame_width=800,        # Map image width
    frame_height=500,       # Map image height
    xlabel="Longitude",     # X Axis Label
    ylabel="Latitude",      # Y Axis Label
    title="Map of Select Cities with Humidity shown by size of spot"  # Title of Map
)

# Display the map
map_of_cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Define ideal weather conditions
min_temp = 21
max_temp = 27
max_wind_speed = 4.5
max_cloudiness = 0

# Filter the DataFrame based on the ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > min_temp) & 
    (city_data_df['Max Temp'] < max_temp) & 
    (city_data_df['Wind Speed'] < max_wind_speed) & 
    (city_data_df['Cloudiness'] == max_cloudiness)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,tolanaro,-25.0319,46.9987,23.70,64,0,2.52,MG,1721128523
6,hamilton,39.1834,-84.5333,24.98,73,0,3.60,US,1721128523
22,kailua-kona,19.6406,-155.9956,23.98,89,0,4.12,US,1721128526
26,bethel,41.3712,-73.4140,23.39,93,0,0.45,US,1721128526
70,tshabong,-26.0500,22.4500,21.43,16,0,2.77,BW,1721128530


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a DataFrame for hotels
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column for hotel names
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()



,City,Country,Lat,Lng,Humidity,Hotel Name
1,tolanaro,MG,-25.0319,46.9987,64,
6,hamilton,US,39.1834,-84.5333,73,
22,kailua-kona,US,19.6406,-155.9956,89,
26,bethel,US,41.3712,-73.4140,93,
70,tshabong,BW,-26.0500,22.4500,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key,    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
hamilton - nearest hotel: North Vista Manor
kailua-kona - nearest hotel: Kona Seaside Hotel
bethel - nearest hotel: Hampton Inn Danbury
tshabong - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
sao felix do xingu - nearest hotel: Hotel Terraço
gulf shores - nearest hotel: Beachside Resort
altamira - nearest hotel: Hotel Vale do Xingu
westport - nearest hotel: Norwalk Inn and Conference Center
perry - nearest hotel: Hampton Inn & Suites
gurupi - nearest hotel: Via Norte
acarau - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
grevenbroich - nearest hotel: Hotel Sonderfeld
alta - nearest hotel: Scandic Alta
greenville - nearest hotel: Quality Inn
benjamin constant - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,tolanaro,MG,-25.0319,46.9987,64,Hôtel Mahavokey
6,hamilton,US,39.1834,-84.5333,73,North Vista Manor
22,kailua-kona,US,19.6406,-155.9956,89,Kona Seaside Hotel
26,bethel,US,41.3712,-73.4140,93,Hampton Inn Danbury
70,tshabong,BW,-26.0500,22.4500,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Remove cities where no hotels were found
cleaned_hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

# Configure the map plot
map_of_hotels = cleaned_hotel_df.hvplot.points(
    "Lng",                  # Longitude points
    "Lat",                  # Latitude points
    geo=True,               # Plot on a geographic coordinate system
    tiles="OSM",            # Use Open Street Maps
    color="City",           # Different colours for each city
    alpha=0.5,              # Sets Transparency at 50%
    frame_width=700,        # Map image width
    frame_height=500,       # Map image height
    xlabel="Longitude",     # X Axis Label
    ylabel="Latitude",      # Y Axis Label
    hover_cols=['Hotel Name', 'Country'],  # Additional information in hover
    title="Map of Hotels for Perfect Holiday"  # Title of Map
)

# Display the map
map_of_hotels


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)